In [1]:
import qcodes as qc
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
import qcodes.dataset.experiment_container as exc
import dash
import dash_core_components as dcc
import dash_html_components as html
from qcodes.dataset.data_set import load_by_id
from qcodes.dataset.data_export import get_data_by_id
from qcodes import load_experiment
from dash.dependencies import Input, Output, State
from plotly import tools
import plotly.io as pio
from plotly import graph_objs as go
from qcodes.instrument_drivers.rohde_schwarz import SGS100A
from time import sleep
from qcodes.dataset.measurements import Measurement
from scipy.optimize import curve_fit
import datetime
import scipy.fftpack
from scipy import signal
from scipy.signal import find_peaks
import pdfkit
from dash.exceptions import PreventUpdate

pyqtgraph plotting not supported, try "from qcodes.plots.pyqtgraph import QtPlot" to see the full error


In [2]:
configuration = qc.config
print(f'Using config file from {configuration.current_config_path}')
configuration['core']['db_location'] = './2019-07-08-frays-final.db'
print(f'Database location: {configuration["core"]["db_location"]}')
qc.initialise_database()

Using config file from //anaconda3/lib/python3.7/site-packages/qcodes/config/qcodesrc.json
Database location: ./2019-07-08-frays-final.db


In [3]:
#exc.experiments()

In [4]:
def normalizeCounts(countArray, num):
    reb_counts = np.squeeze(countArray)/np.mean(np.sort(np.squeeze(countArray))[-num:])
    return reb_counts

In [5]:
def plotdata(expt, files, normalize, plotcurrent=0, nPi = []): 
    "Attribute - expt - can either be 'counting' or 'odmr' or 'pulsedodmr' or 'rabi' or 'ramsey' or 'spinecho' or 'doubleecho' or 'nmr'.\
    If value of plotcurrent is 1 then it will plot the current data. If value of normalize is 1 then it will normalize 'odmr' and 'doubleecho' signal."
    if plotcurrent == 1:        
        filesize = np.size(files) + 1
    else:
        filesize = np.size(files)  
        
    plotfun = go.Figure()    
    for i in range(filesize):
        if plotcurrent == 1 and i == filesize-1:
            Data2 = exc.load_last_experiment()
            Data2 = Data2.last_data_set()
        else:    
            Data2 = exc.load_by_id(files[i])
        if expt == 'spinecho':
            x2 = 2*np.squeeze(Data2.get_data('Time'))
            y2 = np.squeeze(Data2.get_data('Rebased_Counts'))
            plotfun.layout = go.Layout(xaxis=dict(title='Time (ns)'),yaxis=dict(title='Counts'),title='Spin Echo')                  
        elif expt == 'doubleecho':
            if nPi == [] or nPi[i] == 1:
                x2 = 2*np.squeeze(Data2.get_data('Time'))
            else:
                x2 = nPi[i]*np.squeeze(Data2.get_data('Time'))
            y2ms0 = np.squeeze(Data2.get_data('Act_Counts'))
            y2ms1 = np.squeeze(Data2.get_data('Ref_Counts'))
            y2 = y2ms0 - y2ms1   
            plotfun.layout = go.Layout(xaxis=dict(title='Time (ns)'),yaxis=dict(title='Counts'),title='Spin Echo Double Measure')
            if normalize == 1:
                y2 = (y2 + max(y2))/(2*max(y2))
        elif expt == 'nmr':
            x2 = np.squeeze(Data2.get_data('Time'))
            y2ms0 = np.squeeze(Data2.get_data('Act_Counts'))
            y2ms1 = np.squeeze(Data2.get_data('Ref_Counts'))
            y2 = y2ms0 - y2ms1   
            plotfun.layout = go.Layout(xaxis=dict(title='Time (ns)'),yaxis=dict(title='Counts'),title='NMR')
            if normalize == 1:
                y2 = (y2 + max(y2))/(2*max(y2))
        elif expt == 'odmr':
            x2 = np.squeeze(Data2.get_data('Frequency'))
            y2 = np.squeeze(Data2.get_data('Counts'))
            plotfun.layout = go.Layout(xaxis=dict(title='Frequency (Hz)'),yaxis=dict(title='Counts'),title='ODMR')
            if normalize == 1:
                y2 = normalizeCounts(Data2.get_data('Counts'),50)
                layout = go.Layout(xaxis=dict(title='Frequency (Hz)'),yaxis=dict(title='Normalized Counts'),title='ODMR')
        elif expt == 'pulsedodmr':
            x2 = np.squeeze(Data2.get_data('Frequency'))
            y2 = np.squeeze(Data2.get_data('Rebased_Counts'))   
            plotfun.layout = go.Layout(xaxis=dict(title='Frequency (Hz)'),yaxis=dict(title='Counts'),title='Pulsed ODMR')
        elif expt == 'g2':
            x2 = np.squeeze(Data2.get_data('Time'))
            y2 = np.squeeze(Data2.get_data('Norm_Counts'))
            plotfun.layout = go.Layout(xaxis=dict(title='Time dif'), yaxis=dict(title='Normalised Counts'), title='g2 Dip')
        else:
            x2 = np.squeeze(Data2.get_data('Time'))
            y2 = np.squeeze(Data2.get_data('Rebased_Counts'))     
            plotfun.layout = go.Layout(xaxis=dict(title='Time (ns)'),yaxis=dict(title='Counts'),title='Rabi')
        if plotcurrent == 1 and i == filesize-1:
            plotfun.add_scatter(x = x2, y = y2, name = 'Recent Data', mode='lines+markers') 
        else:       
            plotfun.add_scatter(x = x2, y = y2, name = files[i], mode='lines+markers') 
    return plotfun

In [6]:
def lorentzian(x, x0, a0, g, amp):
    denom = (x - x0)**2 + (0.5*g)**2
    num = 0.5*g
    frac = a0 - (num/denom) * (amp)/np.pi
    return frac

In [7]:
def sinedamp(x,a,c,f,t):
    fun = a*np.cos(2*np.pi*f*x)*np.exp(-1*(x/t)) + c
    return fun

In [8]:
def stretchedexp(x,a,c,k,t):
    fun = a*np.exp(-1*(x/t)**k) + c
    return fun

In [9]:
def fitlorentzian(plotfun, expt='pulsedodmr', file=[], lb=None,ub=None):
    "Fitting parameters are ODMR frequency in GHz, Width in GHz, and Amplitude. \
    Attribute - expt - can either be 'odmr' or 'pulsedodmr'"
    if np.size(file) != 0:
        Data = exc.load_by_id(file[0])
    else:
        lastExperiment = exc.load_last_experiment()
        Data = lastExperiment.last_data_set()
    xaxis = 'Frequency'
    x1 = np.squeeze(Data.get_data(xaxis))
    
    if expt == 'odmr':
        yaxis = 'Counts'
        y1 = normalizeCounts(Data.get_data(yaxis),50)
    elif expt == 'pulsedodmr':
        y1 = np.squeeze(Data.get_data('Rebased_Counts'))  
        
    index = np.argmin(y1)
    freq = x1[index]/1e9
    
    if (lb and ub) == None:
        lb = [freq-0.01, 0.95, 0.007, 0.0005]
        ub = [freq+0.01, 1.1, 0.02, 0.005]
        
    popt, pcov = curve_fit(lorentzian, x1/1e9, y1,bounds=(lb, ub))
    fitname = 'Fit' + ', Resonance @ ' + str(round((popt[0]),3)) + ' GHz'
    plotfun.add_scatter(x = x1 , y = lorentzian(x1/1e9,*popt), name = fitname) 
    return popt

In [10]:
def fitsinedamp(plotfun=None,file=[],expt='spinecho',lb=None,ub=None):   
    "Fitting parameters are Amplitude, Baseline Offset, Oscillation Frequency in GHz, and Decay Time in ns.\
    Attribute - expt - can either be 'rabi' or 'spinecho' or 'doubleecho'"
    if np.size(file) != 0:
        Data = exc.load_by_id(file[0])
    else:
        lastExperiment = exc.load_last_experiment()
        Data = lastExperiment.last_data_set()
    xaxis = 'Time'
    
    
    if expt == 'rabi':
        x1 = np.squeeze(Data.get_data(xaxis))
        y1 = np.squeeze(Data.get_data('Rebased_Counts'))
    elif expt == 'spinecho':
        x1 = 2*np.squeeze(Data.get_data(xaxis))
        y1 = np.squeeze(Data.get_data('Rebased_Counts'))
    elif expt == 'doubleecho':
        x1 = 2*np.squeeze(Data.get_data(xaxis))
        y1ms0 = np.squeeze(Data.get_data('Act_Counts'))
        y1ms1 = np.squeeze(Data.get_data('Ref_Counts'))
        y1 = y1ms0 - y1ms1
        
    ampMin = 0.8*(y1.max()-y1.min())/2
    ampMax = 1.2*(y1.max()-y1.min())/2
    boMin = 0.4*(y1.max()+y1.min())
    boMax = 0.6*(y1.max()+y1.min())
    freq = 1/(x1[y1.argmin()]*2)
    freqMin = freq*0.7
    freqMax = freq*1.3
    if (lb and ub) == None:
        lb = [ampMin, boMin, freqMin, 0]
        ub = [ampMax, boMax, freqMax, 1e5]
#     print(f'low bounds are: {lb}')
#     print(f'upper bounds are: {ub}')
    popt, pcov = curve_fit(sinedamp, x1, y1, bounds=(lb,ub))
    yval = sinedamp(x1,*popt)
    fitname = 'Fit' + ', \u0394 = ' + str(round((popt[0]*2*100),1)) + ' %' + ', \u03C0 = ' +  str(round((0.5/popt[2]),1)) + ' ns'
    if plotfun is not None:
        plotfun.add_scatter(x = x1 , y = yval, name = fitname,line=dict(shape='spline')) 
    return popt, pcov

In [11]:
def find_T2(plotfun,file=[],expt='doubleecho',threshold = 0.45, lb=[0,0,0,0],ub=[2,3,3,5e6], revivals=0):   
    "Fitting parameters are Amplitude, Baseline Offset, Power of Stretched Exponential, and Decay Time in ns.\
    Attribute - expt - can either be 'spinecho' or 'doubleecho'.\
    threshold is required for peak detection"
    
    if np.size(file) != 0:
        Data = exc.load_by_id(file[0])
    else:
        lastExperiment = exc.load_last_experiment()
        Data = lastExperiment.last_data_set()
        
    x1 = 2*np.squeeze(Data.get_data('Time'))
    if expt == 'spinecho':
        y1 = np.squeeze(Data.get_data('Rebased_Counts'))  
    elif expt == 'doubleecho':
        y1ms0 = np.squeeze(Data.get_data('Act_Counts'))
        y1ms1 = np.squeeze(Data.get_data('Ref_Counts'))
        y1 = y1ms0 - y1ms1 
        y1 = (y1 + max(y1))/(2*max(y1))
    if revivals == 1:
        peaks, _= find_peaks(y1,height=threshold,distance=6, width=3)
        x0 = np.array([0])
        y0 = np.array([y1[0]])
        xpeaks = np.concatenate([x0, x1[peaks]], axis=0)
        print(xpeaks)
        ypeaks = np.concatenate([y0, y1[peaks]], axis=0)
        print(ypeaks)
    else:
        xpeaks = x1
        ypeaks = y1
    
    popt, pcov = curve_fit(stretchedexp, xpeaks, ypeaks, bounds=(lb,ub))
    yval = stretchedexp(xpeaks,*popt)
    
    fitname = 'Fit, T2 = ' + str(round((popt[3]/1e3),1)) + ' \u03BCs' 
    
    #plotfun.add_scatter(x = xpeaks, y = ypeaks, mode='markers', name = 'Detected Peaks',marker=dict(color='red', size=10, opacity=0.5)) 
    plotfun.add_scatter(x = xpeaks , y = yval, name = fitname, line=dict(shape='spline'), mode='lines')
    
    return popt, pcov

In [12]:
def fouriertransform(file=[],expt='spinecho'):
    "Attribute - expt - can either be 'rabi' or 'ramsey' or 'spinecho' or 'doubleecho'"
    layout = go.Layout(xaxis=dict(title='Frequency (GHz)'),yaxis=dict(title='Amplitude'),title = 'Fourier Transform of a ' + expt.capitalize() + ' signal')
    freqPlot = go.Figure(layout=layout)
    if np.size(file) != 0:
        Data = exc.load_by_id(file[0])
    else:
        lastExperiment = exc.load_last_experiment()
        Data = lastExperiment.last_data_set()
        
    xaxis = 'Time'
    if expt == 'rabi' or expt == 'ramsey':
        x1 = np.squeeze(Data.get_data(xaxis))
        y1 = np.squeeze(Data.get_data('Rebased_Counts'))
    elif expt == 'spinecho':
        x1 = 2*np.squeeze(Data.get_data(xaxis))
        y1 = np.squeeze(Data.get_data('Rebased_Counts'))
    elif expt == 'doubleecho':
        x1 = 2*np.squeeze(Data.get_data(xaxis))
        y1ms0 = np.squeeze(Data.get_data('Act_Counts'))
        y1ms1 = np.squeeze(Data.get_data('Ref_Counts'))
        y1 = y1ms0 - y1ms1
    elif expt == 'nmr':
        x1 = np.squeeze(Data.get_data(xaxis))
        y1ms0 = np.squeeze(Data.get_data('Act_Counts'))
        y1ms1 = np.squeeze(Data.get_data('Ref_Counts'))
        y1 = y1ms1 - y1ms0
        
    step = x1[1] - x1[0]
        
    fourTrans = np.fft.fft(y1)
    freqs = np.fft.fftfreq(y1.shape[-1], step)
    fourTransReal = fourTrans.real
    freqPlot.add_scatter(x = freqs, y = fourTransReal,line=dict(shape='spline'), mode='lines')
    return freqPlot

In [13]:
experimentsList = [exp.name for exp in exc.experiments()]
types = [{'label':'Counting', 'value': 'counting'},
         {'label':'ODMR', 'value': 'odmr'},
         {'label':'Pulsed ODMR', 'value': 'pulsedodmr'},
         {'label':'Rabi', 'value': 'rabi'},
         {'label':'Ramsey', 'value': 'ramsey'},
         {'label':'Spin Echo', 'value': 'spinecho'},
         {'label':'Double Echo', 'value': 'doubleecho'},
         {'label':'NMR', 'value': 'nmr'}]
dataNeeded = {'odmr': ['Frequency','Counts'], 
              'pulsedodmr': ['Frequency','Rebased_Counts'], 
              'spinecho': ['Time','Rebased_Counts'],
              'doubleecho': ['Time','Act_Counts','Ref_Counts'],
              'nmr': ['Time','Act_Counts','Ref_Counts'],
              'counting': ['Time','Rebased_Counts'],
              'rabi': ['Time','Rebased_Counts'],
              'ramsey':['Time','Rebased_Counts']}
analysisTypes = {'odmr': [['Lorentzian','lorentz']], 
                 'pulsedodmr': [['Lorentzian','lorentz']],
                 'spinecho': [['Stretched Exponential','exponent'],['Fourier Transform','fourier']],
                 'doubleecho': [['Stretched Exponential','exponent'],['Fourier Transform','fourier']],
                 'nmr': [['Fourier Transform','fourier']],
                 'counting': [['Not Available!','none']],
                 'rabi': [['Damped Sine','sine']],
                 'ramsey': [['Fourier Transform','fourier']]}
lastSet = (qc.load_last_experiment()).last_data_set()
all_runs = [{'label': str(load_by_id(a)).split('@')[0], 'value': a} for a in range(1,lastSet.run_id)]

In [14]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.config.suppress_callback_exceptions = True

app.layout = html.Div([
    dcc.Store(id='experiment-data', storage_type='session', data = None),
    dcc.Store(id='graph-data', storage_type='session', data = None),
    dcc.Tabs(id="tabs", value='tab-1', children=[dcc.Tab(label='Graph', value='tab-1'),
                                                 dcc.Tab(label='Analyse', value='tab-2'),
    ]),
    html.Div(id='tabs-content')],  
    style={'columnCount': 1})

@app.callback(Output('tabs-content', 'children'),
              [Input('tabs', 'value'), Input('graph-data', 'data')])
def render_content(tab, data):
    if data == None:
        if tab == 'tab-1':
            return html.Div([
                html.H2(children='Graph Data'),
                html.Div([
                    html.Label('1) Experiment'), 
                    dcc.Dropdown(id='experiment', 
                         options=[{'label': experimentsList[o], 'value': o+1} 
                                  for o in range((len(experimentsList)))]
                                ),
                    html.Label('2) Runs'), 
                    dcc.Dropdown(id='runs', multi=True),
                    html.Label('3) Data Type'),
                    dcc.Dropdown(id='data-type', options = types),
                    html.P(id='type-err', style={'color': 'red'})],
                    style={'width': '48%', 'display': 'inline-block'}), 
                html.Div([
                    html.Label('4) Extra Runs (Optional)'), 
                    dcc.Dropdown(id='extra-runs',options = all_runs, multi=True),
                    html.P(id='run-err', style={'color': 'red'}),
                    html.Label('5) Normalise?'), 
                    dcc.RadioItems(id='normalise', options = [{'label': 'Yes', 'value': 1}, 
                                                              {'label': 'No', 'value': 0}], value = 1),
                    html.Button(id='submit-button', children='Plot!')], 
                    style={'width': '48%', 'float': 'right', 'display': 'inline-block'}),
            html.Hr(),
            dcc.Graph(style={'height': '700px'},id='graph', config=dict(showSendToCloud=True))
            ])
        elif tab == 'tab-2':
            return html.Div([
                html.H2(children='Analyse Data'),
                html.Div([
                    html.Label('1) Runs'),
                    dcc.Dropdown(id='analysis-runs',options = all_runs, multi=True),
                    html.Label('2) Data Type'),
                    dcc.Dropdown(id='data-type', options = types),html.Label('3) Analysis Type'),
                    dcc.Dropdown(id='analysis-type')],
                    style={'width': '48%', 'display': 'inline-block'}),
                html.Div([
                    html.Label('4) Normalise?'),
                    dcc.RadioItems(id='analysis-normalise', options = [{'label': 'Yes', 'value': 1}, 
                                                              {'label': 'No', 'value': 0}], value = 1),
                    html.Button(id='analysis-submit-button', children='Plot!')], 
                    style={'width': '48%', 'float': 'right', 'display': 'inline-block'}),
                html.Hr(),
                dcc.Graph(style={'height': '700px'},id='analysis-graph') 
            ])
    else:
        if tab == 'tab-1':
            print('there is currently data')
            return html.Div([
                html.H2(children='Graph Data'),
                html.Div([
                    html.Label('1) Experiment'), 
                    dcc.Dropdown(id='experiment', 
                         options=[{'label': experimentsList[o], 'value': o+1} 
                                  for o in range((len(experimentsList)))]
                                , value = data['experiment']),
                    html.Label('2) Runs'), 
                    dcc.Dropdown(id='runs', multi=True, value = data['run']),
                    html.Label('3) Data Type'),
                    dcc.Dropdown(id='data-type', options = types, value = data['type']),
                    html.P(id='type-err', style={'color': 'red'})],
                    style={'width': '48%', 'display': 'inline-block'}), 
                html.Div([
                    html.Label('4) Extra Runs (Optional)'), 
                    dcc.Dropdown(id='extra-runs',options = all_runs, multi=True, value = data['extra-run']),
                    html.P(id='run-err', style={'color': 'red'}),
                    html.Label('5) Normalise?'), 
                    dcc.RadioItems(id='normalise', options = [{'label': 'Yes', 'value': 1}, 
                                                              {'label': 'No', 'value': 0}], value = data['normalise']),
                    html.Button(id='submit-button', children='Plot!')], 
                    style={'width': '48%', 'float': 'right', 'display': 'inline-block'}),
            html.Hr(),
            dcc.Graph(style={'height': '700px'},id='graph', config=dict(showSendToCloud=True), figure = data['graph'])
            ])
        elif tab == 'tab-2':
            return html.Div([
                html.H2(children='Analyse Data'),
                html.Div([
                    html.Label('1) Runs'),
                    dcc.Dropdown(id='analysis-runs',options = all_runs, multi=True),
                    html.Label('2) Data Type'),
                    dcc.Dropdown(id='data-type', options = types),html.Label('3) Analysis Type'),
                    dcc.Dropdown(id='analysis-type')],
                    style={'width': '48%', 'display': 'inline-block'}),
                html.Div([
                    html.Label('4) Normalise?'),
                    dcc.RadioItems(id='analysis-normalise', options = [{'label': 'Yes', 'value': 1}, 
                                                              {'label': 'No', 'value': 0}], value = 1),
                    html.Button(id='analysis-submit-button', children='Plot!')], 
                    style={'width': '48%', 'float': 'right', 'display': 'inline-block'}),
                html.Hr(),
                dcc.Graph(style={'height': '700px'},id='analysis-graph') 
            ])

@app.callback(
    [Output('runs', 'options'),Output('experiment-data','data')],
    [Input('experiment', 'value')],
    [State('experiment-data','data')])
def update_run(selected_experiment, data):
    if selected_experiment is None:
        raise PreventUpdate
    exper = load_experiment(selected_experiment)
    data = data or {'experiment': None, 'run': None, 'type': None, 'extra-run': None, 'normalise': 1, 'graph': None}
    data['experiment'] = selected_experiment
    print('updated run!')
    return [{'label': str(exper.data_set(a+1)).split('@')[0], 
             'value': exper.data_set(a+1).run_id} for a in range(exper.last_counter)], data

@app.callback(
    [Output('graph', 'figure'), Output('type-err','children'), 
     Output('run-err','children'), Output('graph-data','data')],
    [Input('submit-button','n_clicks')],
    [State('runs','value'), State('data-type','value'), 
     State('extra-runs','value'), State('normalise', 'value'), 
     State('experiment-data','data')])
def update_graph(clicks, selected_runs, selected_type, extra_runs, normalised, data):
    if clicks is None:
        raise PreventUpdate
    if selected_runs is None:
        raise PreventUpdate
    for a in selected_runs:
        params = str(load_by_id(a).parameters).split(',')
        for b in dataNeeded[selected_type]:
            if (b in params) is False:
                return dash.no_update, 'Wrong Data Type!', dash.no_update, dash.no_update
    total_runs = selected_runs
    if extra_runs != None:
        for c in extra_runs:
            params_2 = str(load_by_id(c).parameters).split(',')
            for d in dataNeeded[selected_type]:
                if (d in params_2) is False:
                    return dash.no_update, dash.no_update, 'Wrong Run!', dash.no_update
        total_runs = total_runs + extra_runs
    totaldata = plotdata(selected_type, total_runs, normalised)
    data['run'] = selected_runs
    data['type'] = selected_type
    data['extra-run'] = extra_runs
    data['normalise'] = normalised
    data['graph'] = totaldata
    print('updated graph!')
    return totaldata, None, None, data

In [15]:
if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Aug/2019 18:20:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Aug/2019 18:20:35] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1566376284 HTTP/1.1" 200 -
127.0.0.1 - - [30/Aug/2019 18:20:35] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1566376284 HTTP/1.1" 200 -
127.0.0.1 - - [30/Aug/2019 18:20:35] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1566376284 HTTP/1.1" 200 -
127.0.0.1 - - [30/Aug/2019 18:20:35] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=1566376287 HTTP/1.1" 200 -
127.0.0.1 - - [30/Aug/2019 18:20:35] "GET /_dash-component-suites/dash_renderer/dash_renderer.min.js?v=1.0.0&m=1566376284 HTTP/1.1" 200 -
127.0.0.1 - - [30/Aug/2019 18:20:35] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.1.1&m=1566376285 HTTP/1.1" 200 -
127.0.0.1 - - [30/Aug/2019 1

there is currently data


127.0.0.1 - - [30/Aug/2019 18:20:40] "POST /_dash-update-component HTTP/1.1" 200 -


updated run!


...

updated run!


127.0.0.1 - - [31/Aug/2019 12:26:31] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [31/Aug/2019 12:26:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Aug/2019 12:26:34] "POST /_dash-update-component HTTP/1.1" 200 -


updated graph!
there is currently data


127.0.0.1 - - [31/Aug/2019 12:26:35] "POST /_dash-update-component HTTP/1.1" 200 -


updated run!
